## Send a FITS image view to SkyCharts.

The SAMP (Simple Application Messaging Protocol) is a protocol for communication between astronomy applications. It allows different software to share data and commands. In this example, we will send a FITS image view to the SkyCharts application using the Astropy SAMP module.

In [1]:
import pathlib

from astropy.samp import SAMPHubServer, SAMPIntegratedClient
from astropy.table import Table

Create a table of FITS files in the `finder_charts` directory, with their names and full file path formatted as a URL.

In [2]:
files = [(filepath.name.split('.')[0].replace("_", " ") , f'file:///{filepath.absolute()}')
 for filepath in pathlib.Path('finder_charts').glob('**/*.fits')]

fits_files = Table(rows=files, names=('name', 'url'))
fits_files.add_index('name')

fits_files

name,url
str18,str90
Arp 1 DSS2 color,file:////home/chris/Projects/AL_Northern_Arp_Program/finder_charts/Arp_1_DSS2_color.fits
Arp 153 DSS2 color,file:////home/chris/Projects/AL_Northern_Arp_Program/finder_charts/Arp_153_DSS2_color.fits


Start a SAMP server and connect as an integrated client. 

After starting the server, start SkyCharts and make sure it's connected to the SAMP hub.

This will error if there is already a SAMP server running. In this case start only the client

In [8]:
hub = SAMPHubServer(); hub.start()
client = SAMPIntegratedClient(); client.connect()

INFO: Hub set to run with Web Profile support enabled. [astropy.samp.hub]
INFO: Hub started [astropy.samp.hub]


Select a target and broadcast the message to all connected clients.

In [4]:
target = fits_files.loc['Arp 153 DSS2 color']

In [5]:
params = {'url': str(target['url']),
         'name': str(target['name']),}

message = {'samp.mtype': 'image.load.fits',
           'samp.params': params}

client.notify_all(message)

['cli#2']

Shut down the server

In [9]:
client.disconnect(); hub.stop()

INFO: Hub is stopping... [astropy.samp.hub]
INFO: Hub stopped. [astropy.samp.hub]


## SAMP via external applications.

The Astropy SAMP module is not the most convenient way to interact with other applications. It is better suited to automation tasks where you need to send commands or data to other applications without user intervention.

The easiest way to send a FITS image to SkyCharts is to open it in DS9 and send it to SkyCharts from there. Check to see that both applications are connected to the SAMP hub and click file -> SAMP -> image -> skycharts in menu.

![DS9 SAMP](figures/DS9_SAMP.png)